In [2]:
!pip install pathway bokeh

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.models import HoverTool, ResetTool,BBoxTileSource ,BoxZoomTool
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.palettes import Category20

In [4]:
df =pd.read_csv('/content/dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [5]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)


In [6]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00


In [7]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity"  ]].to_csv("parking_stream.csv", index=False)

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int
    SystemCodeNumber: str

In [8]:
table=pw.io.csv.read(
  '/content/parking_stream.csv',
  schema=ParkingSchema,
  mode="streaming"
)

In [9]:
 data= pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)


In [10]:
  fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
     day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [11]:
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=( pw.this.SystemCodeNumber),  # Group by hour AND system  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size hourly window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        SystemCodeNumber = pw.reducers.any(pw.this.SystemCodeNumber),# Include SystemCodeNumber in the reduce output
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        avg_occ=pw.reducers.avg(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        price = 10 +  4* (pw.this.avg_occ / pw.this.cap)
    )
)
pw.io.csv.write(delta_window, "output_stream.csv")

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [16]:

pn.extension()

#  List of system codes (parking lot identifiers)
system_code_list = [
    "BHMBCCMKT01", "BHMBCCTHL01", "BHMEURBRD01", "BHMMBMMBX01",
    "BHMNCPHST01", "BHMNCPNST01", "Broad Street", "Others-CCCPS105a",
    "Others-CCCPS119a", "Others-CCCPS135a", "Others-CCCPS202",
    "Others-CCCPS8", "Others-CCCPS98", "Shopping"
]

#  Define the per-lot plotting function (for one figure at a time)
def price_plotter(source, system_code):
    fig = bokeh.plotting.figure(
        height=300,
        width=800,
        tools=[HoverTool( tooltips=[("Time", "@t{%F %H:%M}"), ("Price", "@Price")],formatters={"@t": "datetime" ,"@Price":"numeral"}),
               ResetTool(), BoxZoomTool()],
        title=f"Price Trend - {system_code}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=5, color="red")
    return fig

#  Collect one plot per parking lot
viz_panels = []

for code in system_code_list:
    # Filter the main Pathway stream by this parking system
    filtered = delta_window.filter(pw.this.SystemCodeNumber == code)

    # Pathway .plot(...) will provide a reactive source
    viz = filtered.plot(lambda source, code=code: price_plotter(source, code), sorting_col="t")

    # Add each plot as a panel item
    viz_panels.append(pn.Column(f"### {code}", viz))

# Combine all into a single scrollable layout
pn.Column(*viz_panels).servable()


Column
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [3] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [4] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [5] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [6] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [7] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [8] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [9] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [10] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [11] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [12] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [13] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)

In [17]:
pw.run()

Output()